# **Low memory use : be careful**

I share this notebook in order to warn about the use of memory reduction because I am facing weirds results.
I am not an expert in Python and dtypes changes, and maybe these are obvious for some of you. 

Sorry if these observations have already been reported, please let me know it in comments.

In [ ]:
import numpy as np 
import pandas as pd 

Here I am loading the dataset version with parquet (https://www.kaggle.com/datasets/robikscube/ubiquant-parquet)

See this notebook for information https://www.kaggle.com/code/camilomx/parquet-format-quickstart

In [ ]:
%%time
train_df = pd.read_parquet("/kaggle/input/ubiquant-parquet/train_low_mem.parquet")

Let's see the describe information.

In [ ]:
train_df.describe()

In [ ]:
train_df.info()

Everything seems OK.

In [ ]:
# Let's create a copy of train_df
train_df_1 = train_df.copy()

In [ ]:
train_df_1.info()

In [ ]:
train_df_1.describe()

Same information in this copy.

Some people (and me) use this function to reduce the memory use of the dataset. 

Reference : https://www.kaggle.com/code/gemartin/load-data-reduce-memory-usage/notebook

In [ ]:
%%time

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024 ** 2

    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

    return df


train_df_1 = reduce_mem_usage(train_df_1)

In [ ]:
train_df_1.info()

Let's have a look at the describe information.

In [ ]:
train_df_1.describe()

As you can see NaN values are appearing and some values such as mean for features f_i have changed a little.

Let's see some other stranges things :

In [ ]:
# Before memory reduction
investments = train_df["investment_id"].nunique()
print("Before memory reduction \nNumber of unique investiment_id : ", investments)

In [ ]:
# After memory reduction
investments = train_df_1["investment_id"].nunique()
print("After memory reduction \nNumber of unique investiment_id : ", investments)

So, be careful when using data memory reduction.